# 智能投資範例

參考連結：

* https://tw.pycon.org/2019/zh-hant/
* https://trends.gb.net/view/HkBIV1ZJKrM
* http://hilpisch.com/pycontw.pdf
* http://hilpisch.com/pycontw.html

In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web

In [ ]:
df = web.DataReader("SPY","yahoo","2017-01-01")

In [ ]:
df.info()

In [ ]:
#df['r'] = df['Adj Close'].pct_change()
df['r'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
df['d'] = np.where(df['r'] > 0, 1, 0)

In [ ]:
df.head()

In [ ]:
df['long'] = df['r']
df['short'] = -df['r']
df['random'] = np.random.choice([-1, 1], len(df)) * df['r']

In [ ]:
lags = 5

In [ ]:
def create_lags(df, mu, std):
    global cols
    cols = []
    df['rs'] = df['r'] - mu
    df['rs'] /= std
    for lag in range(1, lags + 1):
        col = 'lag_{}'.format(lag)
        df[col] = df['rs'].shift(lag)
        cols.append(col)
    df.dropna(inplace=True)

In [ ]:
ld = len(df)
ld

In [ ]:
split = int(ld * 0.6)
val_size = int(ld * 0.1)

In [ ]:
train = df.iloc[:split]
val = train[-val_size:]
train = train[:-val_size]
test = df.iloc[split:].copy()

In [ ]:
mu = train['r'].mean()
std = train['r'].std()
create_lags(train, mu, std)
create_lags(val, mu, std)
create_lags(test, mu, std)

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop, Adam

In [ ]:
np.random.seed(100)
tf.random.set_random_seed(100)

In [ ]:
opt = RMSprop(lr=0.005, rho=0.9, epsilon=None, decay=0.0)
#opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [ ]:
model = Sequential()

model.add(Dense(128, activation='relu', input_shape=(lags,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

In [ ]:
%%time
model.fit(train[cols], train['d'],
          epochs=50, batch_size=32, verbose=False,
          validation_data=(val[cols], val['d']));

In [ ]:
res = pd.DataFrame(model.history.history)

In [ ]:
res.tail(3)

In [ ]:
model.evaluate(test[cols], test['d'])

In [ ]:
model.metrics_names

In [ ]:
test['p'] = model.predict_classes(test[cols])
test['p'] = np.where(test['p'] > 0, 1, -1)

In [ ]:
test['p'].value_counts()

In [ ]:
test['strategy_dnn'] = test['p'] * test['r']

In [ ]:
res_cols = ['strategy_dnn', 'random', 'short', 'long']

In [ ]:
r = test[res_cols].sum().apply(np.exp).sort_values(ascending=False)
r